In [1]:
from os.path import join

from keras.applications import VGG16, InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical

import pandas as pd
import csv
import os
import numpy as np
import json

from matplotlib import pyplot as plt
import sys
#sys.path.append("../data_preparation/")

from batch_generator_no_shuffle import BatchGenerator

from sklearn.metrics import recall_score, precision_score, f1_score

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Define multi batch generator for predicting (shuffle and random have to be set to False)

In [8]:
def multiple_batch_generator(generator_num, **kwargs):
    """A generator to work with multiple inputs models
    
    We create a model with a list of multiple input layers when 
    we use :func:`keras.layers.concatenate`. However,
    :class:`batch_generator.BatchGenerator` returns a single tuple 
    with two arrays, which does not fit to a model with a multiple 
    input layers. Thus, with this generator, we create the necessary 
    input for such models.
    
    Arguments:
        generator_num {int} -- number of generators should be created
        \**kwargs -- See :class:`batch_generator.BatchGenerator`
        
    Yields:
        ([ndarray,...,ndarray], ndarray) -- in the tuple; list contains feature arrays from each generator, array out of the list contains the label set
    """
    generators_list = [BatchGenerator(**kwargs, shuffle=False) for i in range(generator_num)]
    
    while True:
        Xy_list = [gen.next() for gen in generators_list]
        yield [Xy[0] for Xy in Xy_list], Xy_list[0][1]

def remap(predictions):
    ordering = [1, 10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 11, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 12, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 13, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 14, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 15, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 16, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 17, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 19, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 2, 20, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 21, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22, 220, 221, 222, 223, 224, 225, 226, 227, 228, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 6, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 7, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 8, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 9, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
    fixed = np.zeros(predictions.shape)

    for j in range(predictions.shape[0]):
        for i in range(predictions.shape[1]):
            fixed[j][ordering[i]-1] = predictions[j][i]
            
    return fixed

### Load stacked model

In [3]:
model8 = load_model('models/stacked-new-15000_steps.h5')
model8.compile(optimizer='adam', loss='binary_crossentropy')

### Create test predictions

In [15]:
images_path_test = os.path.abspath('data/test/')
y_test = np.zeros((39706,1))
batch_size = 64
test_steps = int(len(y_test)/batch_size)+1
input_num = 3#len(model.input_layers)
test_generator_multi = multiple_batch_generator(generator_num=input_num,
                                               input_dir=images_path_test,
                                               y=y_test,
                                               batch_size=batch_size,
                                               random=False)

In [16]:
test_predictions = remap(model8.predict_generator(test_generator_multi, steps=test_steps, verbose=1))
np.save('stacked_predictions_15000_test', remap(test_predictions))

621/621 [==============================] - 2139s 3s/step


### Create validation predictions

In [6]:
images_path_validation = os.path.abspath('data/validation/')
y_valid = np.zeros((9897,1))
batch_size = 64
valid_steps = int(len(y_valid)/batch_size)+1
input_num = 3

valid_generator_multi8 = multiple_batch_generator(generator_num=input_num,
                                               input_dir=images_path_validation,
                                               y=y_valid,
                                               batch_size=batch_size)


In [7]:
valid_predictions8 = model8.predict_generator(valid_generator_multi8, steps=valid_steps, verbose=1)
np.save('stacked_predictions_15000_validation', valid_predictions8)

155/155 [==============================] - 556s 4s/step
